In [1]:
import nltk
from nltk.tokenize import word_tokenize, wordpunct_tokenize, RegexpTokenizer, sent_tokenize
from nltk.corpus import stopwords
#from nltk.book import *
import re, pprint
import os
import sys
from datetime import datetime
from datetime import date
#from dateutil.parser import parse
#import pattern3
#from word2number import w2n
import glob
import pandas as pd
import datefinder
import calendar
import unicodedata
import datetime
import boto3
import json
import unidecode

comprehend = boto3.client(service_name='comprehend', region_name='eu-west-1')

In [2]:
def remove_punct(sentence):#remove punctuation and \n tokens
##'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    punctuations = ['.',':',',',';','?'
                    ,'!','/','[',']',
                    '(',')','{','}','"','-','|','`','^','~','<','>']
    sentence = [w for w in sentence if w not in punctuations]
#    sent=[w for w in sentence if w.isalnum()]##not effective
    return sentence

In [3]:
def remove_stops(sentence): #Remove stop words
    stop_words = set(stopwords.words('english'))
    sentence = [w for w in sentence if w not in stop_words]
    return sentence

In [4]:
def preprocess(raw_input):
    sentences = sent_tokenize(raw_input)

    sentences = [word_tokenize(sent) for sent in sentences]
    sentences = [remove_punct(sent) for sent in sentences]##remove punctuations
    sentences = [remove_stops(sent) for sent in sentences]##remove stopwords
    #Add pos_tag for sentences
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [5]:
def get_all_numerals(sentence):
    numbers_context = []
    for w in range(0,len(sentence)):
        ##Check for CD as POS tag of each word of the sentence
        if sentence[w][1]=='CD':
            ##push in context. Ensure the context exists irrespective of position of 'CD' within a sentence
            if w > 1:
                numbers_context.append(sentence[w-2])
            if w > 0:
                numbers_context.append(sentence[w-1])
            numbers_context.append(sentence[w])
            if w < len(sentence)-1:
                numbers_context.append(sentence[w+1])
            if w < len(sentence)-2:
                numbers_context.append(sentence[w+2])
    return numbers_context
    

In [6]:
CONTEXT_SIZE = 5##Context size is total
def get_dates(numbers_in_context,raw_input):
    only_words = []
    month_list=calendar.month_name
    counter = 0
    is_a_date = []
    all_dates = []
    
    for m in range(0,int(len(numbers_in_context)/CONTEXT_SIZE)+1):#for all candidate_dates
        
        if m==int(len(numbers_in_context)/CONTEXT_SIZE)+1:
            context_list = numbers_in_context[m*CONTEXT_SIZE:-1]
        else:
            context_list = numbers_in_context[m*CONTEXT_SIZE:m*CONTEXT_SIZE+CONTEXT_SIZE]
        
        only_words = []
        for w in range(0,len(context_list)):
            only_words.append(context_list[w][0])
            
        list_of_wordContexts = " ".join(str(x) for x in only_words)    
        candidate_dates = list(datefinder.find_dates(list_of_wordContexts))
        if not candidate_dates:
            is_a_date.append(0)
            all_dates.append(0)
        else:
            
            for candid in candidate_dates:
            #search for month_name / number in numbers_in_context of that sentence
                match_year = re.search('\\s\d{4}\\s', list_of_wordContexts)
                if match_year:
                    match_month = re.search('\s+'+month_list[candid.month],list_of_wordContexts)                    
                    if not match_month:
                        match_month = re.search('\s+\d{2}\s+', list_of_wordContexts)
                        
                        if not match_month:
                            is_a_date.append(0)
                            all_dates.append(candid)
                        else:
                            is_a_date.append(1)
                            all_dates.append(candid)
                    else:
                        is_a_date.append(1)
                        all_dates.append(candid)                        
                else:    
                    match_full = re.search('\s+(\d{2})(/|-)(\d{2})(/|-|)\d{4}\s+',list_of_wordContexts)
                    if not match_full:
                            match_full = re.search('\s+(\d{2})(/|-)(\d{2})(/|-|)\d{2}\s+',list_of_wordContexts)
                            is_a_date.append(0)
                            all_dates.append(0)
                    else:  
                        is_a_date.append(1) 
                        all_dates.append(candid)
                        
    return all_dates,is_a_date
    

In [7]:
rootdir = "./../data/*.txt"
file_list = glob.glob(rootdir)
print(file_list)

if file_list:
    for file_count in range(0,len(file_list)):
        f= open(file_list[file_count],'r')
        raw_input = f.read()
        raw_input = re.sub(r'\n{2,}', '.\n',raw_input).strip()
        sentences_aws = sent_tokenize(raw_input)
        
        print('**********Calling AWS Comprehend DetectEntities**********')
        aws_name = os.path.splitext(file_list[file_count])[0]+'_AWS_out.json'  
        with open(aws_name) as f:
            output_dump = json.load(f)        
        
        all_dates = []
        count=0
        f = "%B %d, %Y"
        for x in output_dump:
            for i in range(0,len(output_dump[x]["Entities"])):
                if output_dump[x]["Entities"][i]["Type"]=="DATE":
                    #print(unidecode.unidecode(output_dump[x]["Entities"][i]["Text"]))

                    dates_extracted=unidecode.unidecode(output_dump[x]["Entities"][i]["Text"])
                    list_of_wordContexts = "".join(str(x) for x in dates_extracted)
                    print(list_of_wordContexts)
                    candidate_dates = list(datefinder.find_dates(list_of_wordContexts))
                    for candid in candidate_dates:
                        #print(candid)
                        all_dates.append(candid)

                    
#                     new_date=datetime.datetime.strptime(dates_extracted,f).strftime('%m-%d-%Y')
#                     print(new_date)
#                     all_dates.append(new_date)
        print('*********End of AWS Comprehend DetectEntities********\n')

        print('***********Calling Klassifai Contracts_InfoExtract_NLP******** ')
        sentences = preprocess(raw_input)
        numbers_in_context = [get_all_numerals(sent) for sent in sentences]
        for sent in sentences:
            numbers_in_context = get_all_numerals(sent)
            candidate_dates,is_a_date = get_dates(numbers_in_context,sent)
#            if file_count==1:
            for d in range(0,len(is_a_date)):
                if is_a_date[d]:
                    print(candidate_dates[d])        
        print('*********End of Klassifai Contracts_InfoExtract_NLP********\n')
                
if not file_list:
    print("Not finding list. Pls check")

['./../data/d1.txt', './../data/d2.txt', './../data/d3.txt', './../data/d4.txt', './../data/d5.txt']
**********Calling AWS Comprehend DetectEntities**********
January 28, 2009
March 5, 2002
February 26, 2002
March 5, 2002
January 26, 2009
January 26, 2009
June 27, 1871
June 27, 1871
June 27, 1871
June 27, 1871
June 27, 1871
November 18, 1920
November, 1920
June 27, 1871
June 27, 1871
June 27, 1871
January 19, 1977
November 14, 1985
June 27, 1871
June 27, 1871
June 27, 1871
7
June 27, 1871
June 27, 1871
6
January 19, 1977
November 14, 1985
June 27, 1871
December 16, 1879
June 27, 1871
January 27, 1994
4
January 27, 1994
March 9, 1995
May 5, 1995
May 25,
1995
May 31, 1995
October 26, 1994
*********End of AWS Comprehend DetectEntities********

***********Calling Klassifai Contracts_InfoExtract_NLP******** 
2009-01-28 00:00:00
2009-01-28 00:00:00
2002-03-05 00:00:00
2002-02-26 00:00:00
2002-02-26 00:00:00
2018-03-05 00:00:00
2009-01-26 00:00:00
2009-01-26 00:00:00
2009-01-26 00:00:00
2009-